In [ ]:
from os import scandir
import csv
import numpy as np
from math import log2
from z3 import *
import time

In [ ]:
def build_instances():
    instances = {}
    MPC_instances_path = 'MCP_Instances/'
    with scandir(MPC_instances_path) as inst_list:
        for inst in inst_list:
            inst_path = MPC_instances_path + inst.name
            with open(inst_path) as f:
                reader = csv.reader(f,delimiter=' ')
                data = list(reader)
                instances[inst.name] = {}
                m = int(data[0][0])             #couriers
                n = int(data[1][0])             #items
                l = [int(n) for n in data[2]]   #load sizes (of each courier)
                s = [int(n) for n in data[3]]   #item sizes 
                dx = [int(n) for n in data[4]]   #distribution points, x-coordinate
                dy = [int(n) for n in data[5]]   #distribution points, y-coordinate
                instances[inst.name]['m'] = m 
                instances[inst.name]['n'] = n 
                instances[inst.name]['l'] = l 
                instances[inst.name]['s'] = s
                instances[inst.name]['dx']= dx
                instances[inst.name]['dy']= dy
                #distance-matrix, manhattan distance, order n+1
                instances[inst.name]['Distance_Matrix'] = np.array(
                    [[abs(dx[i]-dx[j])+abs(dy[i]-dy[j]) for i in range(n+1)] for j in range(n+1)]) 
    return instances

instances = build_instances()


#function to print solutions:

def print_routes(X, v, D):
    n = len(X)-1
    routes = {}
    distance = {}
    for i in range(n+1):
        if X[n][i]:
            aux = [n,i]
            vehicle_i = v[i].index(True)
            current_node = i
            distance[vehicle_i] = D[n,i]
            while current_node != n:
                if vehicle_i != v[current_node].index(True): #in the solution there's an arc from i to j, but i and j have different couriers assigned
                    print('Some error occurred with vehicle ', vehicle_i)
                for k in range(n+1):
                    if current_node!=k and X[current_node][k]:
                        aux.append(k)
                        distance[vehicle_i] += D[current_node,k]
                        current_node = k
                        break
            routes[vehicle_i] = aux
            
    total_distance = 0
    for v, route in routes.items():
        total_distance += distance[v]
        route_ = [(node+1) % (n+1) for node in route]
        temp = str(route_)
        temp = temp.replace("[","")
        temp = temp.replace("]","")
        temp = temp.replace(", "," => ")
        route = temp
        print(f"Courier {v+1}: {route},   distance: {distance[v]}")
    print("Total distance: ", total_distance)

    return total_distance


#functions to impose constraints

def exactly_k(bool_vars, k):
    return And(AtMost(*bool_vars, k), AtLeast(*bool_vars, k))


def bin_plus_1(a,b): #constraints to impose that b == a+1 in binary notation
    constraints = []
    c = {}
    num_digits = len(a)
    constraints.append(b[0] == Not(a[0]))
    constraints.append(b[1] == Or(And(a[1],Not(a[0])), And(Not(a[1]),a[0])))
    c[1] = a[0]
    for i in range(2,num_digits):
        c[i] = And(a[i-1],c[i-1])
        constraints.append(b[i] == Or(And(a[i],Not(c[i])),And(Not(a[i]),c[i])))
    return And(constraints)
         

In [ ]:
def find_model(inst, upper_limit = None, remaining_time = 300):
    m,n,l,s = instances[inst]['m'], instances[inst]['n'], instances[inst]['l'], instances[inst]['s']
    D = instances[inst]['Distance_Matrix']

    if upper_limit == None:
        upper_limit = int(np.sum(D)/n) #random path total distance
    
    O = SolverFor("QF_BV")
    O.set("timeout", int(remaining_time)*1000+1) #timeout is in milliseconds

    x = [[Bool(f"x_{i}_{j}") for j in range(n+1)] for i in range(n+1)] # x[i][j] : route (i->j) is used
    v = [[Bool(f"v_{i}_{k}") for k in range(m)] for i in range(n)] #vehicle k is assigned to node i
    len_bin = int(log2(n))+1
    u = [[Bool(f"u_{i}_{k}") for k in range(len_bin)] for i in range(n)] #encoding in binary of order index of node i

    #implicit constraint: no routes from any node to itself
    O.add([x[i][i] == False for i in range(n+1)])


    #exactly one arc enters and exactly one leaves each vertex associated with a customer:
    #each row has exactly one True value: from each node leaves exactly one arc
    O.add([exactly_k([x[i][j] for j in range(n+1)], 1) for i in range(n)]) #leave out row n (arcs leaving from depot)

    #each column has exactly one True value: in each node enters exactly one arc
    O.add([exactly_k([x[i][j] for i in range(n+1)], 1) for j in range(n)]) #leave out the last column (arcs entering in depot)


    O.add(AtMost(*[x[n][j] for j in range(n)], m)) # max m arcs leaving from node n+1 (O)
    O.add(AtMost(*[x[i][n] for i in range(n)], m)) # max m arcs entering O


    #each courier starts from depot at most once:
    O.add([AtMost(*[And(v[j][k],x[n][j]) for j in range(n)], 1) for k in range(m)])


    #if x[i][j] = True, i and j have the same assigned vehicle:
    O.add([Implies(x[i][j], v[i][k] == v[j][k]) for k in range(m) for i in range(n) for j in range(n)])


    O.add([exactly_k(v[i], 1) for i in range(n)]) #one and only one vehicle per node


    #for each vehicle, the total load over its route must be smaller than its max load size
    for k in range(m):
        O.add(PbLe([(v[i][k],s[i]) for i in range(n)], l[k]))


    #(x_ij == 1) -> (u_i + 1 == u_j): subtour elimination #0<=ui<=n
    #Since the depot does not get a value of u, it is possible to drive in a circle only if the vehicle starts and ends at the depot.
    for j in range(n):
        for i in range(n):
            if i!=j:
                O.add(Implies(x[i][j],bin_plus_1(u[i],u[j])))


    #obj function smaller than upper_limit:
    O.add(PbLe([(x[i][j], D[i,j]) for j in range(n+1) for i in range(n+1) if i!=j], upper_limit-1))


    ######
    #symmetry breaking constraint 1:
    #start using vehicles from the vehicle with bigger capacity:

    #for i in range(n):
    #    O.add([Implies(v[i][k2],v[i][k1]) for k1 in range(m) for k2 in range(m) if l[k1]>l[k2]])


    #symmetry breaking constraint 2:
    #nodes order: prefer "d -> 1 -> 3 -> 2 ... -> d"  to "d -> ... -> 2 -> 3 -> 1 -> d"
    O.add([Implies(And(x[n][i],x[i][j]),i<j) for i in range(n) for j in range(n) if i!=j])


    #####
    #Now we look for a model satisfying these constraints:
    #####

    start_time = time.time()

    if O.check() == sat:
        model = O.model()
        r = [[ model.evaluate(x[i][j]) for j in range(n+1) ] for i in range(n+1) ]
        ut = [[model.evaluate(v[i][k]) for k in range(m)] for i in range(n)]
        total_distance = print_routes(r,ut, D)

        elapsed_time = time.time() - start_time

        return total_distance, elapsed_time
    else:
        print ("failed to solve")
        return 0, remaining_time

In [ ]:
inst = 'inst01'

upper_limit, elapsed_time = find_model(inst)
remaining_time = 300 - elapsed_time
print("remaining_time: ", remaining_time)

while remaining_time > 0:
    upper_limit -= int(remaining_time * upper_limit / elapsed_time / 1000) 
    print('looking for solutions with total_distance <', upper_limit)
    upper_limit, elapsed_time = find_model(inst, upper_limit, remaining_time)
    remaining_time = remaining_time - elapsed_time
    print("remaining_time: ", remaining_time)

print("time limit exceeded")